In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

In [ ]:
%run othello_game.ipynb
%run othello_ai.ipynb

### Testing code

Der folgende Code dient zum Test, sowie zum Debuggen der oben implementierten Funktionen.

In [ ]:
test_board = GameState()

In [ ]:
alphabeta_ai_make_move(test_board, combined_heuristic)
test_board.board

Mit Hilfe des Pakets `cProfile` lassen sich Statistiken darüber sammeln, wie viel Zeit in welchen Funktionen benötigt wird.

In [ ]:
import cProfile

cProfile.run('alphabeta_ai_make_move(test_board, combined_heuristic)')
test_board.board

Die Funktion `debug_num_visited_states` berechnet für einen Spielzustand den nächsten Zug mit verschiedenen Algorithmen und misst dabei die benötigte Zeit. Zusätzlich werden mit den globalen Variablen `debug_mm_count` und  `debug_ab_count` die Anzahl der überprüften Zustände gezählt. Diese Zahlen geben einen Überblick darüber, wie viele Zweige durch den Algorithmus ausgeschlossen werden konnten und nicht überprüft werden mussten.

In [ ]:
import time


def debug_num_visited_states(state, depth):
    global transposition_table
    global debug_mm_count
    global debug_ab_count
    global debug_pc_count
    global MINIMAX_DEPTH_LIMIT
    global ALPHABETA_DEPTH_LIMIT
    global PROBCUT_DEPTH_LIMIT

    # save old variables
    old_mm_depth = MINIMAX_DEPTH_LIMIT
    old_ab_depth = ALPHABETA_DEPTH_LIMIT
    old_pb_depth = PROBCUT_DEPTH_LIMIT

    # set depth limit for all algorithms
    MINIMAX_DEPTH_LIMIT = depth
    ALPHABETA_DEPTH_LIMIT = depth
    PROBCUT_DEPTH_LIMIT = depth

    # calculate next move with each algorithm and measure time
    """debug_mm_count= 0
    start = time.time()
    minimax_ai_make_move(copy.deepcopy(state), combined_heuristic)
    secs = time.time() - start
    print("Minimax takes", secs, "seconds and checks", debug_mm_count, "substates")
    debug_ab_count= 0
    transposition_table = {}
    start = time.time()
    alphabeta_ai_make_move(copy.deepcopy(state), combined_heuristic)
    secs = time.time() - start
    print("AlphaBeta takes", secs, "seconds and checks", debug_ab_count, "substates")"""
    debug_ab_count = 0
    transposition_table = {}
    start = time.time()
    alphabeta_id_make_move(copy.deepcopy(state), combined_heuristic)
    secs = time.time() - start
    print("AlphaBeta with iterative deepening takes", secs,
          "seconds and checks", debug_ab_count, "substates")
    debug_pc_count = 0
    transposition_table = {}
    start = time.time()
    probcut_ai_make_move(copy.deepcopy(state), combined_heuristic)
    secs = time.time() - start
    print("ProbCut takes", secs, "seconds and checks",
          debug_pc_count, "substates")

    # restore old variables
    MINIMAX_DEPTH_LIMIT = old_mm_depth
    ALPHABETA_DEPTH_LIMIT = old_ab_depth
    PROBCUT_DEPTH_LIMIT = old_pb_depth

In [ ]:
debug_num_visited_states(test_board, 5)

Die Funktion `get_statistics` ist dazu da, mehrere Spiele zu berechnen, in denen zwei KIs gegeneinander spielen und Statistiken darüber zu sammeln. `num` ist die Anzahl der Spiele, die durchgeführt werden sollen. Die weiteren Parameter legen fest, welche KIs und welche Heuristiken für die Spieler verwendet werden sollen. Statistiken werden nach jedem Spiel aktualisiert.

Da das Spielfeld nicht gezeichnet werden soll, werden statt `next_move` die Funktion `next_move_blind` verwendet.

In [ ]:
import ipywidgets


def get_statistics(num, black_ai, black_h, white_ai, white_h):
    status = ipywidgets.widgets.Label()
    display(status)
    result = []
    wins = [0, 0, 0]
    status.value = f'0 / {num} games played, b/d/w: {wins[0]}/{wins[1]}/{wins[2]}'
    try:
        for i in range(num):
            (b, w) = play_game(black_ai, black_h, white_ai, white_h)
            result.append((b, w))
            if b > w:
                wins[0] += 1
            elif w == b:
                wins[1] += 1
            else:
                wins[2] += 1
            status.value = f'{i+1} / {num} games played, b/d/w: {wins[0]}/{wins[1]}/{wins[2]}'
    except KeyboardInterrupt:
        status.value = f'Interrupted: {i} / {num} games played, b/d/w: {wins[0]}/{wins[1]}/{wins[2]}'
    print_statistics(result)


def play_game(black_ai, black_h, white_ai, white_h):
    state = GameState()
    next_move_blind(state, black_ai, white_ai, {
                    BLACK: black_h, WHITE: white_h})
    return count_disks(state, BLACK), count_disks(state, WHITE)


def next_move_blind(state, black_ai, white_ai, heuristics):
    # Check if/which AI is playing
    strat = black_ai if state.turn == BLACK else white_ai
    strat(state, heuristics[state.turn])
    if not state.game_over:
        next_move_blind(state, black_ai, white_ai, heuristics)


def print_statistics(results):
    print(results)

In [ ]:
get_statistics(5, random_ai_make_move, cowthello_heuristic,
               random_ai_make_move, cowthello_heuristic)